This will check the number of registrations per company, this will be used as a new feature.
==============
**Prepared by** : Grej - Jan. 29, 2019

This is the first part of the process as the result of this will be used for the 2_data_prep.py

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
os.chdir(r'C:\Users\User\Documents\Data_Science_Projects\food-expo-attendee-prediction-project')

# encoding = latin-1 was used here due to the characters that are unreadable when using the standard utf-8
gfm = pd.read_csv(r'.\data\gfm.csv', encoding='latin-1')
afs = pd.read_csv(r'.\data\afs.csv', encoding='latin-1')
gfd = pd.read_csv(r'.\data\glf.csv', encoding='latin-1')

In [3]:
# count the number of registrants per company - categories are combined company name + country, country + website and website only
class createCompanyRegistrationCounts():
    def __init__(self, data):
        self.data = data
    def company_reg_counts(self):
        data = self.data
        data = data[['10 Digit Card Number', 'Company Name', 'Country', 'Website']]
        data['country_company'] = data[['Company Name', 'Country']].apply(lambda x: r''.join(x.astype(str)), axis=1).str.lower().str.replace(r' ', r'')
        count_per_company = data[data['Company Name']!=' '].country_company.value_counts().rename_axis('x').reset_index(name='count_per_company')
        data['country_website'] = data[['Website', 'Country']].apply(lambda x: r''.join(x.astype(str)), axis=1).str.lower().str.replace(r' ', r'')
        count_per_comp_website = data[data['Website']!=' '].country_company.value_counts().rename_axis('y').reset_index(name='count_per_comp_website')
        count_per_website = data[data['Website']!=' '].Website.value_counts().rename_axis('z').reset_index(name='count_per_website')
        data = data.merge(count_per_company, left_on = 'country_company', right_on = 'x', how = 'left')
        data = data.merge(count_per_comp_website, left_on = 'country_website', right_on = 'y', how = 'left')
        data = data.merge(count_per_website, left_on = 'Website', right_on = 'z', how = 'left')
        data = data[['10 Digit Card Number', 'count_per_company', 'count_per_comp_website', 'count_per_website']].fillna(1)
        return data

gfm = createCompanyRegistrationCounts(gfm).company_reg_counts()
afs = createCompanyRegistrationCounts(afs).company_reg_counts()
gfd = createCompanyRegistrationCounts(gfd).company_reg_counts()

'''gfm = company_reg_counts(gfm)
afs = company_reg_counts(afs)
gfd = company_reg_counts(gfd)'''

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


'gfm = company_reg_counts(gfm)\nafs = company_reg_counts(afs)\ngfd = company_reg_counts(gfd)'

In [4]:
data = gfm.append(afs, ignore_index=True).append(gfd, ignore_index=True)

In [5]:
data.to_pickle(r'.\data\output\company_reg_count.pkl')